In [1]:
# imports
import keras
from keras.models import Sequential, Model
from keras.layers import Embedding, Dot, Input, dot
import numpy as np
import utils
import random
import scipy.stats as stats
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [62]:
# load data
data = open('../data/cmudict.dict').readlines()

# convert to word/sequence pairs:
# splitted = [i.replace('\n', '').split(' ') for i in data]
splitted = random.sample([i.replace('\n', '').split(' ') for i in data], 2000)
word_seq_pairs = {i[0]: (index, i[1:]) for index, i in enumerate(splitted)}

In [63]:
# # set global variables here
# embedding_dimension=100
# vocab_size = len(word_seq_pairs)
# input_length = 2
# batch_size = 1

# # create model
# # embedding layer input length should be 2 since i'm training on word pairs
# model = Sequential()
# model.add(Embedding(vocab_size, embedding_dimension, input_length=input_length))
# model.compile('rmsprop', 'mse')
# input_array = np.random.randint(1000, size=(batch_size,input_length))
# output_array = model.predict(input_array)
# # print(output_array)
# assert output_array.shape == (batch_size, input_length, embedding_dimension)

# for layer in model.layers:
#     print(layer)
#     print(layer.output_shape)

In [64]:
# set global variables here
embedding_dimension=50
vocab_size = len(word_seq_pairs)
input_length = 2
batch_size = 1
shape = 1

# create model
# embedding layer input length should be 2 since i'm training on word pairs
word1 = Input(shape=(1,))
word2 = Input(shape=(1,))
embedding_layer = Embedding(vocab_size, embedding_dimension, input_length=1)
word1_encoded = embedding_layer(word1)
word2_encoded = embedding_layer(word2)
dot_product = dot([word1_encoded, word2_encoded], axes=-1)
output = dot_product
model = Model(inputs=[word1, word2], outputs=output)
model.compile('rmsprop', 'mse')
for i in model.layers:
    print(i)
    print(i.output_shape)

(None, 1)
(None, 1)
(None, 1, 50)
(None, 1, 1)


In [71]:
words = list(word_seq_pairs.keys())
def data_generator(batch_size):
    while 1:
#         indices = random.sample(words, 2*batch_size)
        [word1_tuple, word2_tuple] = [word_seq_pairs[i] for i in random.sample(words, 2)]
        dist = utils.get_rhyme_similarity(word1_tuple[1], word2_tuple[1])
        input1 = np.array([word1_tuple[0]])
        input2 = np.array([word1_tuple[0]])
        output = np.array([dist]).reshape((1,1,1))
#         print(input1.shape)
#         print(input2.shape)
#         print(output.shape)
        yield ([input1, input2], output)

# next(data_generator())
model.fit_generator(data_generator(100), steps_per_epoch=10000, epochs=10)

Epoch 1/10
10000/10000 [==============================] - 33s - loss: 0.0118    
Epoch 2/10
10000/10000 [==============================] - 27s - loss: 0.0101    
Epoch 3/10
10000/10000 [==============================] - 25s - loss: 0.0099    
Epoch 4/10
10000/10000 [==============================] - 28s - loss: 0.0096    
Epoch 5/10
10000/10000 [==============================] - 35s - loss: 0.0092    
Epoch 6/10
10000/10000 [==============================] - 33s - loss: 0.0092    
Epoch 7/10
10000/10000 [==============================] - 32s - loss: 0.0096    
Epoch 8/10
10000/10000 [==============================] - 27s - loss: 0.0094    
Epoch 9/10
10000/10000 [==============================] - 26s - loss: 0.0096    
Epoch 10/10
10000/10000 [==============================] - 25s - loss: 0.0099    


In [70]:
import pandas as pd

# get the weights
weights = embedding_layer.get_weights()
weight_matrix = np.vstack(weights)

# get the words that correspond with the weights ids
word_to_weight_dict = {word: weight for word, weight in zip(words, weights)}

# turn into dataframe
df = pd.DataFrame(weight_matrix.T, columns=words)

# make similarity function
def get_similarity(array1, array2):
#     print(type(array1.get_values()))
#     print(array1.get_values())
    return np.sqrt(np.sum((array1.get_values() - array2.get_values())**2))

from scipy.spatial.distance import pdist, squareform
guy = df.values.T
dist = squareform(pdist(guy))
dist[dist<.1]=99
dist_df = pd.DataFrame(data=dist, columns = words, index = words)
# print(dist_df.head())
dist_df
inter = np.argwhere(dist_df.values<.2)
pairs = [(dist_df.columns[row[0]], dist_df.columns[row[1]]) for row in inter]
for i in pairs:
    print(i)


('zavada', 'sitzes')
('zavada', 'shum')
('zavada', 'ung')
('zavada', 'do')
('zavada', 'baggage(2)')
('zavada', 'eljer')
('zavada', 'fiveash')
('zavada', 'biome')
('zavada', 'mabe')
('zavada', 'bored')
('zavada', 'wormser')
('zavada', 'shabby')
('zavada', "ag's")
('zavada', 'herbster')
('zavada', 'baye')
('zavada', 'eiji')
('zavada', 'shown')
('zavada', 'baucom')
('zavada', 'troy')
('zavada', 'flickered')
('zavada', 'zhu')
('zavada', 'bertch')
('zavada', 'gros')
('zavada', 'bugging')
('zavada', 'shue')
('zavada', 'choy')
('zavada', 'chevy')
('zavada', 'charisma')
('zavada', 'biby')
('zavada', 'thao')
('zavada', 'yiddish')
('zavada', 'hysong')
('zavada', 'shoults')
('zavada', 'giannotti')
('zavada', 'kellow')
('zavada', 'chesher')
('zavada', 'addie')
('zavada', 'hop')
('zavada', 'kowalke')
('zavada', 'mannheim')
('caged', 'hairgrove')
('caged', 'parcell')
('caged', 'shum')
('caged', 'ung')
('caged', 'sugarman')
('caged', 'such')
('caged', 'avid(2)')
('caged', 'do')
('caged', 'pedretti')
